In [23]:
import sympy as sp

COMMUTATIVE = False
Astab, D = sp.symbols(r"\tilde{A} D", commutative=COMMUTATIVE)
B, C = sp.symbols('B C', commutative=COMMUTATIVE)
I = 1 #sp.Symbol('I', commutatative=COMMUTATIVE)
J = sp.Matrix([
    [0, B],
    [C, D]
])
J

Matrix([
[0, B],
[C, D]])

In [27]:
sp.simplify((-B * D **-1 * C) ** -1 * B)

-(B*D**(-1)*C)**(-1)*B

In [14]:
Jstab = sp.Matrix([
    [Astab, B],
    [C, D]
])
Jstab

Matrix([
[\tilde{A}, B],
[        C, D]])

In [15]:
1 / I

1.0

In [16]:
Lstab = sp.Matrix([
    [I, 0],
    [C * Astab **-1, I]
])
Lstab_inv = Lstab.inv()
Lstab_inv

Matrix([
[           1, 0],
[-C/\tilde{A}, 1]])

In [17]:
SD = D - C * Astab ** -1 * B
SD_inv = SD ** -1
# SD_inv = sp.Symbol('S_D^{-1}', commutative=COMMUTATIVE)
Diag_stab = sp.Matrix([
    [Astab, 0],
    [0, SD_inv ** -1]
])
Diag_stab_inv = Diag_stab.inv()
Diag_stab_inv

Matrix([
[1/\tilde{A},                      0],
[          0, 1/(-B*C/\tilde{A} + D)]])

In [18]:
Ustab = sp.Matrix([
    [I, Astab ** -1 * B],
    [0, I]
])
Ustab_inv = Ustab.inv()
Ustab_inv

Matrix([
[1, -B/\tilde{A}],
[0,            1]])

In [21]:
sp.simplify(Ustab_inv * Diag_stab_inv * Lstab_inv * J)

Matrix([
[         B*C/(B*C - D*\tilde{A}), 0],
[-C*\tilde{A}/(B*C - D*\tilde{A}), 1]])

In [22]:
sp.simplify(J * Ustab_inv * Diag_stab_inv * Lstab_inv)

Matrix([
[B*C/(B*C - D*\tilde{A}), -B*\tilde{A}/(B*C - D*\tilde{A})],
[                      0,                                1]])

# Apply it to the original matrix 
## Appxoimate inv:

In [185]:
Jstab_inv = Diag_stab_inv * Lstab_inv # Lower
Jstab_inv = Ustab_inv * Diag_stab_inv # Upper
Jstab_inv = Ustab_inv * Diag_stab_inv * Lstab_inv # Full

Jstab_inv_approx = Jstab_inv
Jstab_inv_approx

Matrix([
[\tilde{A}**(-1) + \tilde{A}**(-1)*B*(-C*\tilde{A}**(-1)*B + D)**(-1)*C*\tilde{A}**(-1), -\tilde{A}**(-1)*B*(-C*\tilde{A}**(-1)*B + D)**(-1)],
[                                   -(-C*\tilde{A}**(-1)*B + D)**(-1)*C*\tilde{A}**(-1),                    (-C*\tilde{A}**(-1)*B + D)**(-1)]])

In [187]:
Jstab_inv_approx = Jstab_inv.subs(Astab ** -1, -C ** -1 * D * B ** -1)
Jstab_inv_approx

Matrix([
[-C**(-1)*D*B**(-1)/2, C**(-1)/2],
[           B**(-1)/2, D**(-1)/2]])

## Left (works):

In [229]:
left = Jstab_inv_approx * J
left

Matrix([
[        1/2, 0],
[D**(-1)*C/2, 1]])

In [233]:
left.inv()

Matrix([
[         2, 0],
[-D**(-1)*C, 1]])

In [242]:
left.inv() * Jstab_inv_approx * J

Matrix([
[1, 0],
[0, 1]])

## Right (doesn't work):

In [235]:
right = J * Jstab_inv_approx
right

Matrix([
[1/2, B*D**(-1)/2],
[  0,           1]])

In [238]:
M_inv = right.inv()
M_inv

Matrix([
[2, -B*D**(-1)],
[0,          1]])

In [240]:
J * Jstab_inv_approx * M_inv 

Matrix([
[1, 0],
[0, 1]])

## What does this $M^{-1}$ do?

In [207]:
M = M_inv.inv()
M

Matrix([
[1/2, B*D**(-1)/2],
[  0,           1]])

In [211]:
Jstab_inv_approx 

Matrix([
[-C**(-1)*D*B**(-1)/2, C**(-1)/2],
[           B**(-1)/2, D**(-1)/2]])

In [217]:
J

Matrix([
[1, 0],
[0, 1]])

$M$ seems to be a bridge between $\tilde{J}$ and $J$:

$
    J = M \cdot \tilde{J}
$

$
    J^{-1} = \tilde{J}^{-1} \cdot M^{-1} 
$

In [219]:
Jstab_inv_approx * M_inv

Matrix([
[-C**(-1)*D*B**(-1), C**(-1)],
[           B**(-1),       0]])

In [227]:
(M * Jstab).subs(Astab, -B * D **-1 * C)

Matrix([
[0, B],
[C, D]])

In [223]:
J.inv(method='LU')

Matrix([
[-B**(-1)*D*C**(-1), C**(-1)],
[           B**(-1),       0]])

In [ ]:
class Minv:
    def __init__(self, J55_inv, J45):
        self.J55_inv = J55_inv
        self.J45 = J45
        nd = J55_inv.shape[1] + J45.shape[1]
        self.shape = nd, nd

    def dot(self, x):
        sep = self.J45.shape[0]
        x4 = x[:sep]
        x5 = x[sep:]
        res4 = 2 * x4 - self.J45 @ self.J55_inv.dot(x5)
        return np.concatenate([res4, x5])